In [163]:
#libraries
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd

# Step 1: Get the list of job urls

In [184]:
r = requests.get('https://boards.greenhouse.io/embed/job_board?for=coursera', auth=('user', 'pass'))
r.headers['content-type']

'text/html; charset=utf-8'

In [185]:
r.text

'<!DOCTYPE html>\n<html lang="en">\n<head prefix="og: http://ogp.me/ns#">\n<script type="text/javascript">var _rollbarConfig = {"accessToken":"333f38f9343644929613c6ff7e965f21","captureUncaught":true,"payload":{"environment":"production"},"hostBlockList":["linkedin.com"],"checkIgnore":function(isUncaught, args, payload) {\n  try {\n    var isPhantomJS = window.navigator.userAgent.match(/phantomjs/i);\n    var isSplash= window.navigator.userAgent.match(/splash version/i);\n    var firefox = window.navigator.userAgent.match(/(firefox)\\/(\\w+)/i); // JSON, must escape slash\n\n    var isFirefoxLessThan50;\n    if (firefox) {\n      var majorVersion = Number(firefox[2]);\n      isFirefoxLessThan50 = majorVersion < 50\n    }\n\n    return Boolean(isPhantomJS) || Boolean(isSplash) || Boolean(isFirefoxLessThan50);\n  } catch (e) {\n    return true;\n  }\n}\n};</script>\n<script type="text/javascript">!function(r){var e={};function o(n){if(e[n])return e[n].exports;var t=e[n]={i:n,l:!1,exports

In [186]:
soup = BeautifulSoup(r.text,'html.parser')
soup.find('section').a.get('href')

'https://about.coursera.org/careers/jobs?gh_jid=4645401004'

In [189]:
url = []
for item in soup.find_all('section'):
    url.append(item.a.get('href'))

uniq_url = list(set(url))
uniq_url

['https://about.coursera.org/careers/jobs?gh_jid=4614435004',
 'https://about.coursera.org/careers/jobs?gh_jid=4718868004',
 'https://about.coursera.org/careers/jobs?gh_jid=4687783004',
 'https://about.coursera.org/careers/jobs?gh_jid=4645401004',
 'https://about.coursera.org/careers/jobs?gh_jid=4659828004',
 'https://about.coursera.org/careers/jobs?gh_jid=4703409004']

# Step 2: Job data

In [197]:
r2 = requests.get('https://boards.greenhouse.io/embed/job_app?for=coursera&token=4645401004', auth=('user', 'pass'))

In [198]:
def dict_job(get_request):  
  '''
  get_request: send a GET request to specific URL

  return a dictionary of all necessary data for a single job URL
  '''
  job_dict = {}
  soup = BeautifulSoup(get_request.text,'html.parser')
  job_dict['Title'] = soup.find('h1',{'class':'app-title'}).text #job title
  job_dict['Company'] = soup.find('span',{'class':'company-name'}).text.replace(' at','').strip('\n \n') #company
  job_dict['Location'] = soup.find('div',{'class':'location'}).text.strip('\n \n') #location
  job_dict['Description'] = soup.find_all('ul')[0].text.strip().split('\n') #job description
  job_dict['Basic Qualification'] = soup.find_all('ul')[1].text.strip().split('\n') #basic qualification
  job_dict['Preferred Qualification'] = soup.find_all('ul')[2].text.strip().split('\n') #preferred qualification

  return job_dict

In [199]:
dict_job(r2)

{'Title': 'Content Strategy Manager, Computer Science & Technology',
 'Company': 'Coursera',
 'Location': 'United States',
 'Description': ['Identify in-demand jobs and skills in Computer Science & Tech, and translate them into relevant content and credentials to meet Comp Sci/Info Tech learners’ needs',
  'Consult with companies and universities (via Partnerships Managers) on strategic content opportunities in Computer Science & Tech',
  'Curate and analyze Tech catalogs',
  'Use data and domain knowledge to ensure the overall health of Computer Science & Tech at Coursera',
  'Present internally and externally to knowledge area experts and people without a background in Computer Science & Tech on your recommendations'],
 'Basic Qualification': ["Bachelor's degree in Computer Science, Software Engineering or other tech related discipline",
  '5+ years of experience with a breadth of topics related to Computer Science & Technology',
  'Demonstrated history in data analysis or research, 

# Step 3 - Get the full job data for each job

In [192]:
token_url = []
for i in range(len(uniq_url)):
  token_url.append(re.search("[0-9]+", uniq_url[i]).group())

token_url

['4614435004',
 '4718868004',
 '4687783004',
 '4645401004',
 '4659828004',
 '4703409004']

In [220]:
def get_job_data(root_url,url_arr):
  '''
  root_url: the root string of the job url that does not contain the token
  url_arr: list of unique job tokens scraped from Coursera Job Search webpage

  return the list of all necessary for all jobs on Coursera Job Search webpage
  '''
  result = []
  for token in range(len(token_url)): 
    req = requests.get(root + token_url[token], auth=('user', 'pass'))
    result.append([(key, value) for key, value in dict_job(req).items()])
  
  return result

In [221]:
root = 'https://boards.greenhouse.io/embed/job_app?for=coursera&token='
get_job_data(root,token_url)

[[('Title', 'Engineering Manager'),
  ('Company', 'Coursera'),
  ('Location', 'Bulgaria'),
  ('Description',
   ['Mentoring: Acting as a role model, aiding in career development conversations, inspiring engineers, providing feedback.',
    'Hiring: Building a vision of team skill sets, identifying key growth areas, managing the hiring pipeline.',
    'Advising: Providing technical suggestions to engineers as they grow into engineering leaders in their product work.',
    'Product enablement: Acting as a curator of a vision, discussing needs with stakeholders, prioritizing, driving forward on key initiatives, running sprint planning meetings, and standups.',
    'Platform leadership: balancing effective product feature delivery against managing tech debt',
    'Mediating: Investigating interpersonal issues, facilitating crucial conversations.']),
  ('Basic Qualification',
   ['Bachelors in computer science or relevant degree OR 3+ years of experience as an engineering IC',
    'Leadersh